# Bronze → Silver: Data Exploration & Basic Cleaning

**Purpose:** Explore Bronze data and perform basic cleaning for Silver layer

**What this notebook does:**
- Explore Bronze data (distributions, nulls, outliers)
- Basic cleaning (remove nulls, fix types, deduplicate)
- Export to Silver table

**What this notebook does NOT do:**
- Feature engineering (that's in Feast)
- Aggregations (that's in Feast)
- Model training (that's in MLflow)

## Flow:
Bronze (raw) → **This Notebook** (clean) → Silver (clean) → Label Studio (annotate) → Gold (primitives) → Feast (features)

In [23]:
!pip install -q boto3 pandas trino pyarrow numpy

In [24]:
import os
import pandas as pd
from trino.dbapi import connect
from datetime import datetime

In [25]:
# Configuration
TRINO_HOST = os.getenv('TRINO_HOST', 'trino')
TRINO_PORT = int(os.getenv('TRINO_PORT', '8080'))
ICEBERG_CATALOG = os.getenv('ICEBERG_CATALOG', 'iceberg_dev')

BRONZE_TABLE = f'{ICEBERG_CATALOG}.bronze.fraud_transactions'
SILVER_TABLE = f'{ICEBERG_CATALOG}.silver.fraud_transactions'

print(f"Trino: {TRINO_HOST}:{TRINO_PORT}")
print(f"Catalog: {ICEBERG_CATALOG}")
print(f"Bronze: {BRONZE_TABLE}")
print(f"Silver: {SILVER_TABLE}")

Trino: exp-trino:8080
Catalog: iceberg_dev
Bronze: iceberg_dev.bronze.fraud_transactions
Silver: iceberg_dev.silver.fraud_transactions


In [26]:
# Connect to Trino
conn = connect(
    host=TRINO_HOST,
    port=TRINO_PORT,
    user='trino',
    catalog=ICEBERG_CATALOG,
    schema='bronze'
)

print("✓ Connected to Trino")

✓ Connected to Trino


## 1. Explore Bronze Data

In [27]:
# Load Bronze data (with chunking support for 1M+ scalability)
# For large datasets, use chunksize parameter

# Option 1: Quick exploration with LIMIT (default)
CHUNK_SIZE = int(os.getenv("NOTEBOOK_CHUNK_SIZE", "0"))  # 0 = use LIMIT

if CHUNK_SIZE > 0:
    # Streaming mode for 1M+ records
    print(f"Loading Bronze data in chunks of {CHUNK_SIZE}...")
    chunks = []
    for chunk in pd.read_sql(f"SELECT * FROM {BRONZE_TABLE}", conn, chunksize=CHUNK_SIZE):
        chunks.append(chunk)
        print(f"  Loaded chunk: {len(chunk)} rows")
    df = pd.concat(chunks, ignore_index=True) if chunks else pd.DataFrame()
else:
    # Quick exploration mode with LIMIT
    df = pd.read_sql(f"SELECT * FROM {BRONZE_TABLE} LIMIT 20000", conn)

print(f"Loaded {len(df)} rows from Bronze")
df.head()

/tmp/ipykernel_20632/131217751.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {BRONZE_TABLE} LIMIT 20000", conn)


Loaded 12999 rows from Bronze


,transaction_id,customer_id,transaction_amount,transaction_date,payment_method,product_category,quantity,customer_age,customer_location,device_used,ip_address,shipping_address,billing_address,is_fraudulent,account_age_days,transaction_hour,source_lakefs_commit,source_file,ingested_at,ingestion_date
0,TX-20260119113423-000000-0000,CUST-000000,14217.67,2026-01-13 22:57:06,debit_card,Grocery,1,38,"London, GB",mobile,148.50.181.163,"Toronto, CA","London, GB",1,170,22,c25b5a71d39fd1bbac95f3cee07468aeb7f2a0ba975e51...,demo/demo/fraud_transactions/2026_01_20_176888...,2026-01-20 05:17:16.907137,2026-01-20
1,TX-20260114132851-000000-0000,CUST-000000,14217.67,2026-01-08 22:57:06,debit_card,Grocery,1,38,"London, GB",mobile,148.50.181.163,"Toronto, CA","London, GB",1,170,22,48a75c1d8b5c0c8595ca457656522a5f01f8fae386b409...,demo/demo/fraud_transactions/2026_01_14_176841...,2026-01-15 07:06:16.468896,2026-01-15
2,TX-20260119111512-000000-0000,CUST-000000,14217.67,2026-01-13 22:57:06,debit_card,Grocery,1,38,"London, GB",mobile,148.50.181.163,"Toronto, CA","London, GB",1,170,22,6cd093b23428e33418d740442c25c1192e415ec6c6205d...,demo/demo/fraud_transactions/2026_01_19_176882...,2026-01-19 13:19:32.084517,2026-01-19
3,TX-20260114132851-000000-0001,CUST-000000,1892.02,2026-01-09 12:21:28,wallet,Fashion,1,38,"London, GB",mobile,148.50.181.175,"London, GB","London, GB",0,171,12,48a75c1d8b5c0c8595ca457656522a5f01f8fae386b409...,demo/demo/fraud_transactions/2026_01_14_176841...,2026-01-15 07:06:16.468896,2026-01-15
4,TX-20260114132851-000000-0002,CUST-000000,1253.29,2026-01-11 13:23:38,debit_card,Luxury,1,40,"London, GB",mobile,148.50.181.70,"London, GB","London, GB",0,173,13,48a75c1d8b5c0c8595ca457656522a5f01f8fae386b409...,demo/demo/fraud_transactions/2026_01_14_176841...,2026-01-15 07:06:16.468896,2026-01-15


In [28]:
# Basic stats
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12999 entries, 0 to 12998
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   transaction_id        12999 non-null  object        
 1   customer_id           12999 non-null  object        
 2   transaction_amount    12999 non-null  float64       
 3   transaction_date      12999 non-null  datetime64[ns]
 4   payment_method        12999 non-null  object        
 5   product_category      12999 non-null  object        
 6   quantity              12999 non-null  int64         
 7   customer_age          12999 non-null  int64         
 8   customer_location     12999 non-null  object        
 9   device_used           12999 non-null  object        
 10  ip_address            12999 non-null  object        
 11  shipping_address      12999 non-null  object        
 12  billing_address       12999 non-null  object        
 13  is_fraudulent   

In [29]:
# Check for nulls
null_counts = df.isnull().sum()
print("Null counts:")
print(null_counts[null_counts > 0])

Null counts:
Series([], dtype: int64)


In [30]:
# Check fraud distribution
print("Fraud distribution:")
print(df['is_fraudulent'].value_counts())
print(f"\nFraud rate: {df['is_fraudulent'].mean():.2%}")

Fraud distribution:
is_fraudulent
0    8563
1    4436
Name: count, dtype: int64

Fraud rate: 34.13%


In [31]:
# Amount distribution
df['transaction_amount'].describe()

count    12999.000000
mean      3091.171772
std       3732.585888
min         50.000000
25%       1118.590000
50%       1931.800000
75%       3275.830000
max      43979.840000
Name: transaction_amount, dtype: float64

In [32]:
# Check duplicates
duplicates = df.duplicated(subset=['transaction_id']).sum()
print(f"Duplicate transaction_ids: {duplicates}")

Duplicate transaction_ids: 0


## 2. Basic Cleaning

In [33]:
# Remove records with null critical fields
df_clean = df[
    df['transaction_id'].notna() &
    df['customer_id'].notna() &
    df['transaction_amount'].notna() &
    df['transaction_date'].notna()
].copy()

print(f"Removed {len(df) - len(df_clean)} rows with null critical fields")
print(f"Remaining: {len(df_clean)} rows")

Removed 0 rows with null critical fields
Remaining: 12999 rows


In [34]:
# Fill nulls in non-critical fields with defaults
df_clean['payment_method'] = df_clean['payment_method'].fillna('unknown')
df_clean['product_category'] = df_clean['product_category'].fillna('unknown')
df_clean['quantity'] = df_clean['quantity'].fillna(1)
df_clean['customer_age'] = df_clean['customer_age'].fillna(0)
df_clean['customer_location'] = df_clean['customer_location'].fillna('unknown')
df_clean['device_used'] = df_clean['device_used'].fillna('unknown')
df_clean['account_age_days'] = df_clean['account_age_days'].fillna(0)
df_clean['transaction_hour'] = df_clean['transaction_hour'].fillna(0)

print("✓ Filled nulls in non-critical fields")

✓ Filled nulls in non-critical fields


In [35]:
# Remove duplicates (keep first)
df_clean = df_clean.drop_duplicates(subset=['transaction_id'], keep='first')
print(f"After deduplication: {len(df_clean)} rows")

After deduplication: 12999 rows


In [36]:
# Verify no nulls in critical fields
critical_nulls = df_clean[['transaction_id', 'customer_id', 'transaction_amount', 'transaction_date']].isnull().sum()
assert critical_nulls.sum() == 0, "Critical fields still have nulls!"
print("✓ No nulls in critical fields")

✓ No nulls in critical fields


## 3. Export to Silver

In [37]:
# Create Silver schema
cursor = conn.cursor()
cursor.execute(f"CREATE SCHEMA IF NOT EXISTS {ICEBERG_CATALOG}.silver")
print("✓ Silver schema ready")

✓ Silver schema ready


In [38]:
# Create Silver table
create_sql = f"""
CREATE TABLE IF NOT EXISTS {ICEBERG_CATALOG}.silver.fraud_transactions (
    -- ═══════════════════════════════════════════════════════════════
    -- Core Transaction Fields (from Bronze/Source)
    -- ═══════════════════════════════════════════════════════════════
    transaction_id VARCHAR,
    customer_id VARCHAR,
    transaction_amount DOUBLE,  -- Match Bronze float64
    transaction_date TIMESTAMP(6),
    payment_method VARCHAR,
    product_category VARCHAR,
    quantity BIGINT,  -- Match Bronze int64
    customer_age BIGINT,
    customer_location VARCHAR,
    device_used VARCHAR,
    ip_address VARCHAR,
    shipping_address VARCHAR,
    billing_address VARCHAR,
    is_fraudulent BIGINT,
    account_age_days BIGINT,
    transaction_hour BIGINT,
    
    -- ═══════════════════════════════════════════════════════════════
    -- Bronze Layer Metadata (lineage tracking)
    -- ═══════════════════════════════════════════════════════════════
    source_file VARCHAR,
    source_lakefs_commit VARCHAR,
    ingested_at TIMESTAMP(6),  -- When Bronze ingested
    ingestion_date DATE,
    
    -- ═══════════════════════════════════════════════════════════════
    -- Silver Layer Processing
    -- ═══════════════════════════════════════════════════════════════
    silver_processed_at TIMESTAMP(6),
    
    -- ═══════════════════════════════════════════════════════════════
    -- Label Studio / Human Review
    -- ═══════════════════════════════════════════════════════════════
    review_status VARCHAR,  -- 'pending', 'in_review', 'reviewed'
    reviewed_label BIGINT,  -- Human-verified label (0/1)
    annotations VARCHAR,
    annotated_by VARCHAR,
    annotated_at TIMESTAMP(6)
)
WITH (partitioning = ARRAY['ingestion_date'])
"""

cursor.execute(create_sql)
print("✓ Silver table created")

# Verify
cursor.execute(f"DESCRIBE {SILVER_TABLE}")
print(f"✓ {len(cursor.fetchall())} columns")

✓ Silver table created
✓ 26 columns


In [39]:
# ═══════════════════════════════════════════════════════════════════════════════
# FIXED: Bronze → Silver MERGE (with partition pruning for 1M+ scalability)
# ═══════════════════════════════════════════════════════════════════════════════

# Count before
cursor.execute(f"SELECT COUNT(*) FROM {BRONZE_TABLE}")
bronze_count = cursor.fetchone()[0]
cursor.execute(f"SELECT COUNT(*) FROM {SILVER_TABLE}")
silver_before = cursor.fetchone()[0]
print(f"Bronze: {bronze_count:,} rows")
print(f"Silver before: {silver_before:,} rows")

# MERGE: Fixed to match actual Bronze schema
# Added partition pruning (ingestion_date) for better performance at 1M+ scale
merge_sql = f"""
MERGE INTO {SILVER_TABLE} AS target 
USING (
    SELECT DISTINCT
        -- Core fields from Bronze
        transaction_id,
        customer_id,
        transaction_amount,
        transaction_date,
        COALESCE(payment_method, 'unknown') as payment_method,
        COALESCE(product_category, 'unknown') as product_category,
        COALESCE(quantity, 1) as quantity,
        COALESCE(customer_age, 0) as customer_age,
        COALESCE(customer_location, 'unknown') as customer_location,
        COALESCE(device_used, 'unknown') as device_used,
        COALESCE(ip_address, 'unknown') as ip_address,
        COALESCE(shipping_address, 'unknown') as shipping_address,
        COALESCE(billing_address, 'unknown') as billing_address,
        is_fraudulent,
        account_age_days,
        transaction_hour,
        
        -- Bronze metadata
        source_file,
        source_lakefs_commit,
        ingested_at,
        ingestion_date
        
    FROM {BRONZE_TABLE}
    WHERE transaction_id IS NOT NULL
      AND customer_id IS NOT NULL
      AND transaction_amount IS NOT NULL
      AND transaction_date IS NOT NULL
      -- Partition pruning for incremental loads
      AND ingestion_date >= CURRENT_DATE - INTERVAL '7' DAY
) AS source
ON target.transaction_id = source.transaction_id
   AND target.ingestion_date >= CURRENT_DATE - INTERVAL '7' DAY

WHEN NOT MATCHED THEN
    INSERT (
        transaction_id,
        customer_id,
        transaction_amount,
        transaction_date,
        payment_method,
        product_category,
        quantity,
        customer_age,
        customer_location,
        device_used,
        ip_address,
        shipping_address,
        billing_address,
        is_fraudulent,
        account_age_days,
        transaction_hour,
        source_file,
        source_lakefs_commit,
        ingested_at,
        ingestion_date,
        silver_processed_at,
        review_status,
        reviewed_label,
        annotations,
        annotated_by,
        annotated_at
    )
    VALUES (
        source.transaction_id,
        source.customer_id,
        source.transaction_amount,
        source.transaction_date,
        source.payment_method,
        source.product_category,
        source.quantity,
        source.customer_age,
        source.customer_location,
        source.device_used,
        source.ip_address,
        source.shipping_address,
        source.billing_address,
        source.is_fraudulent,
        source.account_age_days,
        source.transaction_hour,
        source.source_file,
        source.source_lakefs_commit,
        source.ingested_at,
        source.ingestion_date,
        CURRENT_TIMESTAMP,
        'pending',
        NULL,
        NULL,
        NULL,
        NULL
    )
"""

print("Executing MERGE with partition pruning (1 snapshot)...")
cursor.execute(merge_sql)
print("✓ MERGE complete")

# Count after
cursor.execute(f"SELECT COUNT(*) FROM {SILVER_TABLE}")
silver_after = cursor.fetchone()[0]
print(f"✓ Silver after: {silver_after:,} rows")
print(f"✓ New records: {silver_after - silver_before:,}")

Bronze: 12,999 rows
Silver before: 11,036 rows
Executing MERGE with partition pruning (1 snapshot)...
✓ MERGE complete
✓ Silver after: 12,999 rows
✓ New records: 1,963


In [40]:
# Verify Silver table
count_query = f"SELECT COUNT(*) FROM {SILVER_TABLE}"
cursor.execute(count_query)
silver_count = cursor.fetchone()[0]
print(f"\nSilver table has {silver_count} rows")


Silver table has 12999 rows


In [41]:
# ═════════════════════════════════════════════════════════════════════════════
# Verify snapshot count
# ═════════════════════════════════════════════════════════════════════════════
try:
    cursor.execute(f"""
        SELECT COUNT(*) 
        FROM {ICEBERG_CATALOG}.silver."fraud_transactions$snapshots"
        WHERE committed_at >= CURRENT_DATE
    """)
    today_snapshots = cursor.fetchone()[0]
    print(f"\\n📊 Snapshots created today: {today_snapshots}")
    print(f"   Expected: 1 per notebook run (not 10!)")
    
    # Show recent snapshots
    cursor.execute(f"""
        SELECT snapshot_id, committed_at, operation
        FROM {ICEBERG_CATALOG}.silver."fraud_transactions$snapshots"
        ORDER BY committed_at DESC
        LIMIT 5
    """)
    print(f"\\n📝 Recent snapshots:")
    for row in cursor.fetchall():
        print(f"   {row[1]}: {row[2]} (snapshot {row[0]})")
except Exception as e:
    print(f"Could not query snapshots: {e}")

\n📊 Snapshots created today: 1
   Expected: 1 per notebook run (not 10!)
\n📝 Recent snapshots:
   2026-01-20 05:24:59.970000+00:00: overwrite (snapshot 1474762575304462843)
   2026-01-19 14:35:58.967000+00:00: overwrite (snapshot 7938298294713540718)
   2026-01-19 14:26:18.652000+00:00: overwrite (snapshot 7381085809395400885)
   2026-01-19 13:30:00.822000+00:00: overwrite (snapshot 4191169207103915603)
   2026-01-15 11:20:43.380000+00:00: overwrite (snapshot 3771614438766913717)


In [42]:
# Sample from Silver
df_silver = pd.read_sql(f"SELECT * FROM {SILVER_TABLE} LIMIT 10", conn)
df_silver.head()

/tmp/ipykernel_20632/3265316248.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_silver = pd.read_sql(f"SELECT * FROM {SILVER_TABLE} LIMIT 10", conn)


,transaction_id,customer_id,transaction_amount,transaction_date,payment_method,product_category,quantity,customer_age,customer_location,device_used,...,source_file,source_lakefs_commit,ingested_at,ingestion_date,silver_processed_at,review_status,reviewed_label,annotations,annotated_by,annotated_at
0,TX-20260120051403-000000-0002,CUST-000000,1253.29,2025-08-09 07:14:06,credit_card,Fashion,1,40,"London, GB",desktop,...,demo/demo/fraud_transactions/2026_01_20_176888...,c25b5a71d39fd1bbac95f3cee07468aeb7f2a0ba975e51...,2026-01-20 05:17:16.907137,2026-01-20,2026-01-20 05:24:59.090,pending,None,None,None,None
1,TX-20260120051403-000000-0008,CUST-000000,2089.52,2025-09-08 04:53:01,cod,Grocery,1,40,"London, GB",desktop,...,demo/demo/fraud_transactions/2026_01_20_176888...,c25b5a71d39fd1bbac95f3cee07468aeb7f2a0ba975e51...,2026-01-20 05:17:16.907137,2026-01-20,2026-01-20 05:24:59.090,pending,None,None,None,None
2,TX-20260120051403-000000-0028,CUST-000000,2580.16,2026-01-08 07:50:41,cod,Luxury,4,39,"London, GB",mobile,...,demo/demo/fraud_transactions/2026_01_20_176888...,c25b5a71d39fd1bbac95f3cee07468aeb7f2a0ba975e51...,2026-01-20 05:17:16.907137,2026-01-20,2026-01-20 05:24:59.090,pending,None,None,None,None
3,TX-20260120051403-000000-0030,CUST-000000,1592.31,2026-01-12 22:30:14,credit_card,Home,1,40,"London, GB",mobile,...,demo/demo/fraud_transactions/2026_01_20_176888...,c25b5a71d39fd1bbac95f3cee07468aeb7f2a0ba975e51...,2026-01-20 05:17:16.907137,2026-01-20,2026-01-20 05:24:59.090,pending,None,None,None,None
4,TX-20260119113423-000000-0003,CUST-000000,1787.65,2026-01-17 09:20:00,cod,Fashion,3,41,"London, GB",mobile,...,demo/demo/fraud_transactions/2026_01_20_176888...,c25b5a71d39fd1bbac95f3cee07468aeb7f2a0ba975e51...,2026-01-20 05:17:16.907137,2026-01-20,2026-01-20 05:24:59.090,pending,None,None,None,None


In [43]:
# Check review status distribution
cursor = conn.cursor()

cursor.execute(f"""
    SELECT review_status, COUNT(*) as count
    FROM {ICEBERG_CATALOG}.silver.fraud_transactions
    GROUP BY review_status
""")
print("Review Status Distribution:")
for row in cursor.fetchall():
    print(f"  {row[0]}: {row[1]:,}")

Review Status Distribution:
  pending: 1,963
  reviewed: 11,036


## 4. Summary

**What we did:**
- ✅ Explored Bronze data (nulls, distributions, duplicates)
- ✅ Basic cleaning (removed nulls, filled defaults, deduped)
- ✅ Exported to Silver table

**Next steps:**
1. Silver → Label Studio for human annotation
2. Annotated data → Gold (primitives only)
3. Gold → Feast (feature engineering with aggregations)
4. Feast → MLflow (training)

**Note:** Feature engineering (rolling windows, cross-features, risk scores) is now done in Feast, not here!

In [44]:
# Close connection
conn.close()
print("✓ Notebook complete")

✓ Notebook complete
